# GPX to CZML

Converting GPX data from Strava to CZML format for visualization CesiumJS.

### Import Libraries

In [5]:
import pandas as pd
import numpy as np
import gpxpy
import json

### Read GPX File

In [6]:
gpx_file = open('../data/Woodstock_NY.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

### Extract Locations, Elevations and Timestamps from GPX file

In [7]:
## See primer on reading GPX data in python here: http://andykee.com/visualizing-strava-tracks-with-python.html

def parse_gpx(gpx_input_file):
    
    lats = []
    lons = []
    elevations = []
    timestamps = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lats.append(point.latitude)
                lons.append(point.longitude)
                elevations.append(point.elevation)
                timestamps.append(point.time)
                   
    output = pd.DataFrame()
    output['latitude'] = lats
    output['longitude'] = lons
    output['elevation'] = elevations
    output['starttime'] = timestamps
    output['stoptime'] = output['starttime'].shift(-1).fillna(method='ffill')
    output['duration'] = (output['stoptime'] - output['starttime']) / np.timedelta64(1, 's') ## duration to seconds
    
    return output

Here's what the GPX data looks like after we parse it:

In [8]:
df = parse_gpx(gpx)
df.head()

,latitude,longitude,elevation,starttime,stoptime,duration
0,42.029884,-74.107225,162.0,2017-06-11 19:51:59,2017-06-11 19:52:03,4.0
1,42.029855,-74.107269,163.0,2017-06-11 19:52:03,2017-06-11 19:52:05,2.0
2,42.029824,-74.107292,163.4,2017-06-11 19:52:05,2017-06-11 19:52:12,7.0
3,42.029788,-74.107308,163.4,2017-06-11 19:52:12,2017-06-11 19:52:20,8.0
4,42.029826,-74.107312,163.4,2017-06-11 19:52:20,2017-06-11 19:52:22,2.0


### Generate CZML Path

In [28]:
def czml_path(df_input, relative_elevation = False):
    results = []
    
    clock = 0
    
    for i in df_input.index:
        results.append(clock)
        results.append(df_input.longitude.ix[i])
        results.append(df_input.latitude.ix[i])
        
        if relative_elevation == True:
            results.append(30) # for use with point = {"heightReference" : "RELATIVE_TO_GROUND"}
        else:
            results.append(df_input.elevation.ix[i])
        
        duration = df_input.duration.ix[(i)]
        clock += duration
        
    return results

# Animate Point with Trailing Path

In [51]:
def moving_point_with_path(df_input, time_mult = 100, path_type="outline"):
    
    czml_output = []

    # Required global variables
    global_id = "document"
    global_name = "Visualizing GPX data from Strava"
    global_version = "1.0"

    # Optional global variables
    global_author = "Will Geary"
    global_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    global_stoptime = str(max(df_input['stoptime'])).replace(" ", "T").replace(".000", "Z")
    global_availability = global_starttime + "/" + global_stoptime    
    
    # Create first element with global variables
    global_element = {
        "id" : global_id,
        "name" : global_name,
        "version" : global_version,
        "author": global_author,
        "clock": {
            "interval": global_availability,
            "currentTime": global_starttime,
            "multiplier": time_mult
        }
    }
    
    # Add first element to output
    czml_output.append(global_element)
    
    ### PATH ###
    
    # Child variable
    child_id = "path"
    child_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_stoptime = str(max(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_availability = child_starttime + "/" + child_stoptime
    
    if path_type == "outline":
        child_element = {
                "id": child_id,

                "availability": child_availability,

                "position": {
                    "epoch": child_starttime,
                    "cartographicDegrees": czml_path(df, relative_elevation=False)
                },

                "path" : {
                    "material" : {
                        "polylineOutline" : {
                            "color" : {
                                "rgba" : [255,255,255, 200]
                            },
                            "outlineColor" : {
                                "rgba" : [0,173,253, 200]
                            },
                            "outlineWidth" : 5
                        }
                    },
                    "width" : 6,
                    "leadTime" : 0,
                    "trailTime" : 100000,
                    "resolution" : 5
                }
            }
        
    elif path_type == "glow":
        child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path(df, relative_elevation=False)
            },

            "path" : {
                "material" : {
                    "polylineGlow" : {
                        "color" : {
                            "rgba" : [0,173,253, 200]
                        },
                        "glowPower" : 0.1
                    }
                },
                "width" : 6,
                "leadTime" : 0,
                "trailTime" : 100000,
                "resolution" : 5
            }
        }
        
    elif path_type == "dash":
        child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path(df, relative_elevation=False)
            },

            "path" : {
                "material" : {
                    "polylineDash" : {
                        "color" : {
                            "rgba" : [0,173,253, 200]
                        }
                    }
                },
                "width" : 6,
                "leadTime" : 0,
                "trailTime" : 100000,
                "resolution" : 5
            }
        }

    czml_output.append(child_element)
    
    
    ### POINT ###
    
    # Child variable
    child_id = "Point"
    child_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_stoptime = str(max(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_availability = child_starttime + "/" + child_stoptime
    
    child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path(df, relative_elevation=True)
            },

            "point": {
                "color": {
                    "rgba": [255, 255, 255, 255]
                },
                "outlineColor": {
                    "rgba": [0,173,253, 255]
                },
                "outlineWidth":6,
                "pixelSize":8,
                "heightReference" : "RELATIVE_TO_GROUND"
            }   
        }

    czml_output.append(child_element)
    
    return czml_output

In [53]:
czml_output_path = moving_point_with_path(df, path_type="outline")
with open('/Users/Will/node_modules/Cesium-1.33/Apps/Data/woodstock.czml', 'w') as outfile:
    json.dump(czml_output_path, outfile)

### Generate CZML Point

In [140]:
def moving_point(df_input, time_mult = 100):
    
    czml_output = []

    # Required global variables
    global_id = "document"
    global_name = "Visualizing GPX data from Strava"
    global_version = "1.0"

    # Optional global variables
    global_author = "Will Geary"
    global_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    global_stoptime = str(max(df_input['stoptime'])).replace(" ", "T").replace(".000", "Z")
    global_availability = global_starttime + "/" + global_stoptime    
    
    # Create first element with global variables
    global_element = {
        "id" : global_id,
        "name" : global_name,
        "version" : global_version,
        "author": global_author,
        "clock": {
            "interval": global_availability,
            "currentTime": global_starttime,
            "multiplier": time_mult
        }
    }
    
    # Add first element to output
    czml_output.append(global_element)

    # Child variable
    child_id = "Point"
    child_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_stoptime = str(max(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_availability = child_starttime + "/" + child_stoptime
    
    child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path
            },

            "point": {
                "color": {
                    "rgba": [255, 255, 255, 200]
                },
                "outlineColor": {
                    "rgba": [255, 0, 0, 200]
                },
                "outlineWidth":8,
                "pixelSize":10,
                "heightReference" : "RELATIVE_TO_GROUND"
            }   
        }

    czml_output.append(child_element)
    
    return czml_output

In [141]:
czml_output = moving_point(df)

In [142]:
with open('/Users/Will/node_modules/Cesium-1.33/Apps/Data/woodstock.czml', 'w') as outfile:
    json.dump(czml_output, outfile)

# Animate Path

In [145]:
def moving_path(df_input, time_mult = 100):
    
    czml_output = []

    # Required global variables
    global_id = "document"
    global_name = "Visualizing GPX data from Strava"
    global_version = "1.0"

    # Optional global variables
    global_author = "Will Geary"
    global_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    global_stoptime = str(max(df_input['stoptime'])).replace(" ", "T").replace(".000", "Z")
    global_availability = global_starttime + "/" + global_stoptime    
    
    # Create first element with global variables
    global_element = {
        "id" : global_id,
        "name" : global_name,
        "version" : global_version,
        "author": global_author,
        "clock": {
            "interval": global_availability,
            "currentTime": global_starttime,
            "multiplier": time_mult
        }
    }
    
    # Add first element to output
    czml_output.append(global_element)

    # Child variable
    child_id = "path"
    child_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_stoptime = str(max(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_availability = child_starttime + "/" + child_stoptime
    
    child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path
            },

            "path" : {
                "material" : {
                    "polylineOutline" : {
                        "color" : {
                            "rgba" : [255, 0, 255, 255]
                        },
                        "outlineColor" : {
                            "rgba" : [0, 255, 255, 255]
                        },
                        "outlineWidth" : 5
                    }
                },
                "width" : 8,
                "leadTime" : 10,
                "trailTime" : 100000,
                "resolution" : 5
            }
        }

    czml_output.append(child_element)
    
    
    
    # Child variable
    child_id = "Point"
    child_starttime = str(min(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_stoptime = str(max(df_input['starttime'])).replace(" ", "T").replace(".000", "Z")
    child_availability = child_starttime + "/" + child_stoptime
    
    child_element = {
            "id": child_id,

            "availability": child_availability,

            "position": {
                "epoch": child_starttime,
                "cartographicDegrees": czml_path
            },

            "point": {
                "color": {
                    "rgba": [255, 255, 255, 255]
                },
                "outlineColor": {
                    "rgba": [255, 0, 0, 255]
                },
                "outlineWidth":8,
                "pixelSize":10,
                "heightReference" : "RELATIVE_TO_GROUND"
            }   
        }

    czml_output.append(child_element)
    
    return czml_output

In [146]:
czml_output_path = moving_path(df)
with open('/Users/Will/node_modules/Cesium-1.33/Apps/Data/woodstock.czml', 'w') as outfile:
    json.dump(czml_output_path, outfile)